In [11]:
import polars as pl
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, text

In [27]:
df = pd.read_parquet('data/video-transcripts.parquet')
df_eval = pd.read_csv('data/eval-raw.csv')

Define the database connection parameters

In [ ]:
# Define the database connection parameters
db_params = {
	'host': 'host',
	'database': 'databse',
	'user': 'user',
	'password': 'password'
}

In [21]:
conn = psycopg2.connect(
	host=db_params['host'],
	database=db_params['database'],
	user=db_params['user'],
	password=db_params['password']
)

In [22]:
# Create a cursor object
cur = conn.cursor()

In [23]:
# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

In [24]:
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:5432/{db_params["database"]}')

In [13]:
csv_files = {
    'eval_csv' : r'F:\github\YouTube-Blog\full-stack-data-science\data-science\data\eval-raw.csv'
}

In [25]:
# Load and display the contents of each CSV file to check
for table_name, file_path in csv_files.items():
    print(f"Contents of '{table_name}' CSV file:")
    df = pd.read_csv(file_path)
    print(df.head(2))  # Display the first few rows of the DataFrame
    print("\n")

Contents of 'eval_csv' CSV file:
             query     video_id
0    ai consulting  INlCLmWlojY
1  fine tuning llm  eC6Hd1hFvos




Push csv to postgreSQL table

In [26]:
# Loop through the CSV files and import them into PostgreSQL
for table_name, file_path in csv_files.items():
    df = pd.read_csv(file_path)
    df.to_sql(table_name, engine, if_exists='replace', index=False)

push parquet to postgreSQL table

In [31]:
df.to_sql("video-transcripts", engine, if_exists='replace', index=False)

83

In [32]:
pd.read_sql_table("video-transcripts", engine, schema='public')

,video_id,datetime,title,transcript
0,03x2oYg9oME,2024-04-25 15:16:00,Data Science Project Management (5-step framew...,this video is part of a larger series on full ...
1,O5i_mMUM94c,2024-04-19 14:05:54,How I’d learned #datascience (if I had to star...,here's how I'd learn data science if I had to ...
2,xm9devSQEqU,2024-04-18 15:59:02,4 Skills You Need to Be a Full-Stack Data Scie...,although it is common to delegate different pa...
3,Z6CmuVEi7QY,2024-04-11 10:00:27,How I'd Learn Data Science (if I started over),when I was first learning data science it was ...
4,INlCLmWlojY,2024-04-04 18:45:00,I Was Wrong About AI Consulting (what I learned),last year I quit my corporate data science job...
...,...,...,...,...
78,MX7ymkYGiZ0,2020-12-21 00:24:45,The Wavelet Transform | Introduction & Example...,hey guys welcome back in this video we'll be t...
79,rPUytg38b6Q,2020-12-04 01:10:36,The Fast Fourier Transform | How does it (actu...,hey guys welcome back in this video we'll be t...
80,mj86XmfOniY,2020-11-15 20:41:53,"Time Series, Signals, & the Fourier Transform ...",yeah hey guys welcome my name is sean i'm a ph...
81,Gwz4zXPeP_Q,2020-11-12 22:58:00,biometricDahboard3 DEMO,n/a
